**Théophane DELBAUFFE - ESILV DIA4**

# Week 2

## Requirements

In [ ]:
from random import seed
import csv
import random as rd
from random import random
from math import exp
from sklearn.metrics import mean_squared_error

## Initialize network

### EX1

In [ ]:
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

### EX2

In [ ]:
seed(1)
network = initialize_network(3,2,2)
for layer in network:
  print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217]}, {'weights': [0.49543508709194095, 0.4494910647887381, 0.651592972722763, 0.7887233511355132]}]
[{'weights': [0.0938595867742349, 0.02834747652200631, 0.8357651039198697]}, {'weights': [0.43276706790505337, 0.762280082457942, 0.0021060533511106927]}]


### EX3

In [ ]:
def activate(weights, inputs):
	activation = weights[-1] # bias
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

### EX4

In [ ]:
def transfer(activation):
  return 1.0 / (1.0 + exp(-activation))

## Forward Propagation

### EX5

In [ ]:
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

### EX6

In [ ]:
seed(1)
network = initialize_network(3,2,2)
output = forward_propagate(network,[2,0,1])
print(output)

[0.7181660053840072, 0.7392260370751094]


### EX7

In [ ]:
def transfer_derivative(output):
	return output * (1.0 - output)

## Error Backpropagation

### EX8

In [ ]:
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))): # we start with the last layer (the output layer because backpropagation)
		layer = network[i]
		errors = list()
  
		if i != len(network)-1: # for hidden and inputs layers
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
    
		else: # first case : when we start with the output layer, the error of each neuron is easily computed
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
    

		for j in range(len(layer)): 
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

### EX9

In [ ]:
seed(1)
network = initialize_network(3,2,2)
forward_propagate(network,[2,0,1])
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217], 'output': 0.7837359639983127, 'delta': -0.0013748786059863865}, {'weights': [0.49543508709194095, 0.4494910647887381, 0.651592972722763, 0.7887233511355132], 'output': 0.9191747246018124, 'delta': -0.0025407285391936397}]
[{'weights': [0.0938595867742349, 0.02834747652200631, 0.8357651039198697], 'output': 0.7181660053840072, 'delta': 0.145359380646418}, {'weights': [0.43276706790505337, 0.762280082457942, 0.0021060533511106927], 'output': 0.7392260370751094, 'delta': -0.05026963236025112}]


# Week 3

## Train Network

### EX10

In [ ]:
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

### EX11

In [ ]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

### EX12

In [ ]:
seed(1)
small_dataset = [[rd.uniform(0,10) for i in range(2)]+[rd.randint(0,1)] for i in range(10)]
small_dataset

[[1.3436424411240122, 8.474337369372327, 0],
 [2.550690257394217, 4.954350870919409, 1],
 [4.722452435761166, 3.7961522332372777, 0],
 [0.9385958677423489, 0.2834747652200631, 1],
 [4.3276706790505335, 7.62280082457942, 0],
 [6.958328667684435, 2.663305604572596, 0],
 [5.911534350013039, 1.0222715811004823, 1],
 [0.30589983033553536, 0.254458609934608, 0],
 [9.391491627785106, 3.8120423768821246, 0],
 [9.690406502940995, 7.258526014465152, 0]]

In [ ]:
n_inputs = 2
n_outputs = 2
n_hidden = 2
l_rate = 0.3
n_epoch = 50

network = initialize_network(n_inputs, n_hidden, n_outputs)
train_network(network, small_dataset, l_rate, n_epoch, n_outputs)

for layer in network:
	print(layer)

>epoch=0, lrate=0.300, error=7.606
>epoch=1, lrate=0.300, error=6.659
>epoch=2, lrate=0.300, error=5.550
>epoch=3, lrate=0.300, error=4.808
>epoch=4, lrate=0.300, error=4.486
>epoch=5, lrate=0.300, error=4.356
>epoch=6, lrate=0.300, error=4.297
>epoch=7, lrate=0.300, error=4.266
>epoch=8, lrate=0.300, error=4.247
>epoch=9, lrate=0.300, error=4.234
>epoch=10, lrate=0.300, error=4.223
>epoch=11, lrate=0.300, error=4.214
>epoch=12, lrate=0.300, error=4.204
>epoch=13, lrate=0.300, error=4.195
>epoch=14, lrate=0.300, error=4.186
>epoch=15, lrate=0.300, error=4.176
>epoch=16, lrate=0.300, error=4.166
>epoch=17, lrate=0.300, error=4.157
>epoch=18, lrate=0.300, error=4.148
>epoch=19, lrate=0.300, error=4.140
>epoch=20, lrate=0.300, error=4.132
>epoch=21, lrate=0.300, error=4.125
>epoch=22, lrate=0.300, error=4.118
>epoch=23, lrate=0.300, error=4.111
>epoch=24, lrate=0.300, error=4.105
>epoch=25, lrate=0.300, error=4.099
>epoch=26, lrate=0.300, error=4.092
>epoch=27, lrate=0.300, error=4.086
>e

## Predict

### EX13

In [ ]:
def predict(network, row):
  outputs = forward_propagate(network, row)
  index = outputs.index(max(outputs)) # we return the index of the max of the outputs variable, which contains the probability assigned to each binary output
  return index

### EX14

In [ ]:
for row in small_dataset:
  preds = predict(network,row)
  print("Reality : ",row[-1]," -- ", preds, " : Prediction")
  # I don't unsterdand why does it not work... Whereas it works very well for the last question

Reality :  0  --  0  : Prediction
Reality :  1  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  1  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  1  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction


## Seed Dataset

### EX15

In [ ]:
'''
from google.colab import files
uploaded = files.upload()
print(uploaded)
'''

'\nfrom google.colab import files\nuploaded = files.upload()\nprint(uploaded)\n'

### EX16

In [ ]:
def load_csv(filename):
  dataset=list()
  with open(filename,'r') as data:
    data_csv=csv.reader(data)
    for row in data_csv:
      dataset.append([float(numbers) for numbers in row[0].split('\t')])
  return dataset

### EX17

In [ ]:
# load
data = load_csv("seeds_dataset.csv")

# normalize
min_max = [[min(column), max(column)] for column in zip(*data)]

for row in data:
  row[-1] = int(row[-1]) - 1
  for i in range(len(row)-1):
	  row[i] = round((row[i] - min_max[i][0]) / (min_max[i][1] - min_max[i][0]),3)

# shuffle
rd.shuffle(data)

# split
train = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

### EX18

In [ ]:
l_rate = 0.3
n_epoch = 50
n_hidden = 5
n_inputs = len(train[0])-1
n_outputs = 3

network = initialize_network(n_inputs, n_hidden, n_outputs)
train_network(network, train, l_rate, n_epoch, n_outputs)

>epoch=0, lrate=0.300, error=145.846
>epoch=1, lrate=0.300, error=113.987
>epoch=2, lrate=0.300, error=109.941
>epoch=3, lrate=0.300, error=102.279
>epoch=4, lrate=0.300, error=90.324
>epoch=5, lrate=0.300, error=78.469
>epoch=6, lrate=0.300, error=70.299
>epoch=7, lrate=0.300, error=65.179
>epoch=8, lrate=0.300, error=61.686
>epoch=9, lrate=0.300, error=58.945
>epoch=10, lrate=0.300, error=56.486
>epoch=11, lrate=0.300, error=54.051
>epoch=12, lrate=0.300, error=51.514
>epoch=13, lrate=0.300, error=48.847
>epoch=14, lrate=0.300, error=46.099
>epoch=15, lrate=0.300, error=43.372
>epoch=16, lrate=0.300, error=40.779
>epoch=17, lrate=0.300, error=38.411
>epoch=18, lrate=0.300, error=36.315
>epoch=19, lrate=0.300, error=34.497
>epoch=20, lrate=0.300, error=32.936
>epoch=21, lrate=0.300, error=31.597
>epoch=22, lrate=0.300, error=30.447
>epoch=23, lrate=0.300, error=29.451
>epoch=24, lrate=0.300, error=28.585
>epoch=25, lrate=0.300, error=27.824
>epoch=26, lrate=0.300, error=27.151
>epoch=

### EX19

In [ ]:
for row in test:
  preds = predict(network,row)
  print("Reality : ",row[-1]," -- ", preds, " : Prediction")

Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  1  --  1  : Prediction
Reality :  2  --  2  : Prediction
Reality :  2  --  2  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  1  --  1  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  2  --  2  : Prediction
Reality :  1  --  1  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  0  --  0  : Prediction
Reality :  2  --  2  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  --  0  : Prediction
Reality :  0  